In [1]:
import os
import numpy as np
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv
from multielo import MultiElo, Player, Tracker
from mktools.get_data import load_data_pd
from alive_progress import alive_it
from mktools.validate_data import validate_bad_uids

load_dotenv()

True

In [3]:
df = load_data_pd(
    sheet_name="data_main",
    sheet_id=os.environ["SHEET_ID"],
    usecols=[
        "UID",
        "SUID",
        "NAME",
        "CHARACTER",
        "MAP",
        "PLACE",
        "PLAYERS",
        "DATE",
        "SEASON",
    ],
)

invalid, valid = validate_bad_uids(df=df, return_valid=True)

In [3]:
valid

,UID,SUID,NAME,CHARACTER,MAP,PLACE,PLAYERS,DATE,SEASON
0,1,1,Cole,Toad,Sherbet Land,4,4,2021-09-20,0
1,1,1,Connor,Yoshi,Sherbet Land,2,4,2021-09-20,0
2,1,1,Cooper,Peach,Sherbet Land,1,4,2021-09-20,0
3,1,1,Triston,Bowser,Sherbet Land,3,4,2021-09-20,0
4,2,1,Cole,Toad,Kalimari Desert,4,4,2021-09-20,0
...,...,...,...,...,...,...,...,...,...
20631,6294,737,Connor,Toad,Toad's Turnpike,3,4,2024-07-17,11
20632,6294,737,Garrett,Peach,Toad's Turnpike,4,4,2024-07-17,11
20633,6295,737,Cooper,Toad,Bowser's Castle,1,3,2024-07-17,11
20634,6295,737,Cole,Yoshi,Bowser's Castle,2,3,2024-07-17,11


In [94]:
rated_seasons = []

for season in alive_it(
    it=valid["SEASON"].unique(),
    total=valid["SEASON"].unique().shape[0],
    theme="classic",
):

    tdf = (
        valid[(valid["PLAYERS"] == 4) & (valid["SEASON"] == season)]
        .drop(columns=["SUID", "CHARACTER", "PLAYERS", "SEASON", "MAP"])
        .sort_values(by=["UID", "PLACE"])
        .copy()
        .reset_index(drop=True)
    )

    pdf = (
        tdf.pivot(index="UID", columns="PLACE", values="NAME")
        .reset_index()
        .rename(columns={1: "1st", 2: "2nd", 3: "3rd", 4: "4th"})
        .merge(tdf[["UID", "DATE"]], on="UID", how="inner", validate="1:m")
        .groupby("UID")
        .first()
        .set_index("DATE")
        .reset_index()
        .rename(columns={"DATE": "date"})
    )

    exp_elo = MultiElo(score_function_base=1.5)
    tracker = Tracker(elo_rater=exp_elo)
    tracker.process_data(pdf)

    rdf = tracker.get_current_ratings()

    rdf["SEASON"] = season

    rated_seasons.append(rdf)

[!                                       ] (!) 0/12 [0%] in 0.0s (0.00/s) 


ValueError: Index contains duplicate entries, cannot reshape

In [95]:
rated_seasons[-1]

IndexError: list index out of range

In [64]:
pd.concat(rated_seasons).sort_values(by=["SEASON", "rank"]).reset_index(drop=True)

,rank,player_id,n_games,rating,SEASON
0,1,Cooper,142,1431.138670,0
1,2,Regan,58,1212.174791,0
2,3,Cole,33,1153.182061,0
3,4,Matt,68,1138.867777,0
4,5,Blake,29,1085.996876,0
...,...,...,...,...,...
235,16,Randy,10,876.515824,11
236,17,Colton,21,872.667841,11
237,18,Domingo,26,840.244082,11
238,19,Antonio,15,839.396319,11


In [66]:
tdf

,UID,NAME,PLACE,DATE
0,1,Cooper,1,2021-09-20
1,1,Connor,2,2021-09-20
2,1,Triston,3,2021-09-20
3,1,Cole,4,2021-09-20
4,2,Cooper,1,2021-09-20
...,...,...,...,...
21641,6288,Regan,3,2024-07-15
21642,6288,Konnor,4,2024-07-15
21643,6289,Regan,1,2024-07-15
21644,6289,Cooper,2,2024-07-15


In [67]:
df

,UID,SUID,NAME,CHARACTER,MAP,PLACE,PLAYERS,DATE,SEASON
0,1,1,Cole,Toad,Sherbet Land,4,4,2021-09-20,0
1,1,1,Connor,Yoshi,Sherbet Land,2,4,2021-09-20,0
2,1,1,Cooper,Peach,Sherbet Land,1,4,2021-09-20,0
3,1,1,Triston,Bowser,Sherbet Land,3,4,2021-09-20,0
4,2,1,Cole,Toad,Kalimari Desert,4,4,2021-09-20,0
...,...,...,...,...,...,...,...,...,...
21641,6288,735,Regan,Yoshi,Moo Moo Farm,3,4,2024-07-15,11
21642,6288,735,Konnor,Bowser,Moo Moo Farm,4,4,2024-07-15,11
21643,6289,735,Regan,Toad,Bowser's Castle,1,3,2024-07-15,11
21644,6289,735,Cooper,D.K.,Bowser's Castle,2,3,2024-07-15,11


In [96]:
msk = (valid["UID"].value_counts().reset_index()["count"] > 4 ) | (valid["UID"].value_counts().reset_index()["count"] < 2)

valid[valid["UID"].isin(valid["UID"].value_counts().index[msk])]

,UID,SUID,NAME,CHARACTER,MAP,PLACE,PLAYERS,DATE,SEASON


In [110]:
tdf = (
    valid[(valid["PLAYERS"] == 4) & (valid["SEASON"] == 0)]
    .drop(columns=["SUID", "CHARACTER", "PLAYERS", "SEASON", "MAP"])
    .sort_values(by=["UID", "PLACE"])
    .copy()
    .reset_index(drop=True)
)

tdf

,UID,NAME,PLACE,DATE
0,1,Cooper,1,2021-09-20
1,1,Connor,2,2021-09-20
2,1,Triston,3,2021-09-20
3,1,Cole,4,2021-09-20
4,2,Cooper,1,2021-09-20
...,...,...,...,...
1559,689,Cole,4,2021-11-10
1560,692,Matt,1,2021-11-10
1561,692,Cooper,2,2021-11-10
1562,692,Blake,3,2021-11-10


In [111]:
tdf["UID"].value_counts()

UID
1      4
390    4
432    4
424    4
423    4
      ..
197    4
193    4
192    4
191    4
692    4
Name: count, Length: 391, dtype: int64

In [107]:
pdf = (
    tdf.pivot(index="UID", columns="PLACE", values="NAME")
    .reset_index()
    .rename(columns={1: "1st", 2: "2nd", 3: "3rd", 4: "4th"})
    .merge(tdf[["UID", "DATE"]], on="UID", how="inner", validate="1:m")
    .groupby("UID")
    .first()
    .set_index("DATE")
    .reset_index()
    .rename(columns={"DATE": "date"})
)

pdf

,date,1st,2nd,3rd,4th
0,2022-01-25,Cooper,Blake,Connor,Jake
1,2022-01-25,Cooper,Connor,Jake,Blake
2,2022-01-25,Cooper,Blake,Jake,Connor
3,2022-01-25,Connor,Cole,Blake,Robert
4,2022-01-27,Luke,Matt,Chandler,Robert
...,...,...,...,...,...
230,2022-03-12,Cole,Blake,Connor,Jake
231,2022-03-13,Cooper,Regan,Cole,Triston
232,2022-03-13,Cooper,Cole,Triston,Regan
233,2022-03-13,Cooper,Regan,Cole,Triston


In [97]:
tdf = (
    valid.drop(columns=["SUID", "CHARACTER", "PLAYERS", "SEASON", "MAP"])
    .sort_values(by=["UID", "PLACE"])
    .copy()
    .reset_index(drop=True)
)

pdf = (
    tdf.pivot(index="UID", columns="PLACE", values="NAME")
    .reset_index()
    .rename(columns={1: "1st", 2: "2nd", 3: "3rd", 4: "4th"})
    .merge(tdf[["UID", "DATE"]], on="UID", how="inner", validate="1:m")
    .groupby("UID")
    .first()
    .set_index("DATE")
    .reset_index()
    .rename(columns={"DATE": "date"})
)

exp_elo = MultiElo(score_function_base=1.5)
tracker = Tracker(elo_rater=exp_elo)
tracker.process_data(pdf)

rdf = tracker.get_current_ratings()

ValueError: Index contains duplicate entries, cannot reshape

In [4]:
tdf = (
    valid[(valid["PLAYERS"] == 4) & (valid["SEASON"] == 11)]
    .drop(columns=["SUID", "CHARACTER", "PLAYERS", "SEASON", "MAP"])
    .sort_values(by=["UID", "PLACE"])
    .copy()
    .reset_index(drop=True)
)

gp = tdf["NAME"].value_counts().reset_index().rename(columns={"count": "GAMES_PLAYED"})

tdf_merge = tdf.merge(gp, how="inner", on="NAME", validate="m:1")

tdf_filter = tdf_merge[tdf_merge["GAMES_PLAYED"] > 15].reset_index(drop=True)

tdf_filter

,UID,NAME,PLACE,DATE,GAMES_PLAYED
0,6002,Blake,1,2024-06-21,29
1,6002,Cooper,2,2024-06-21,144
2,6002,Matt,3,2024-06-21,68
3,6002,Garrett,4,2024-06-21,43
4,6003,Cooper,1,2024-06-21,144
...,...,...,...,...,...
571,6291,Triston,4,2024-07-16,25
572,6294,Cooper,1,2024-07-17,144
573,6294,Cole,2,2024-07-17,35
574,6294,Connor,3,2024-07-17,26


In [5]:
valid["DATE"]

0        2021-09-20
1        2021-09-20
2        2021-09-20
3        2021-09-20
4        2021-09-20
            ...    
20674           NaN
20675           NaN
20676           NaN
20677           NaN
20678           NaN
Name: DATE, Length: 20679, dtype: object

In [8]:
valid["DATE"].tail(50)

20629    2024-07-17
20630    2024-07-17
20631    2024-07-17
20632    2024-07-17
20633    2024-07-17
20634    2024-07-17
20635    2024-07-17
20636           NaN
20637           NaN
20638           NaN
20639           NaN
20640           NaN
20641           NaN
20642           NaN
20643           NaN
20644           NaN
20645           NaN
20646           NaN
20647           NaN
20648           NaN
20649           NaN
20650           NaN
20651           NaN
20652           NaN
20653           NaN
20654           NaN
20655           NaN
20656           NaN
20657           NaN
20658           NaN
20659           NaN
20660           NaN
20661           NaN
20662           NaN
20663           NaN
20664           NaN
20665           NaN
20666           NaN
20667           NaN
20668           NaN
20669           NaN
20670           NaN
20671           NaN
20672           NaN
20673           NaN
20674           NaN
20675           NaN
20676           NaN
20677           NaN
20678           NaN


In [6]:
valid

,UID,SUID,NAME,CHARACTER,MAP,PLACE,PLAYERS,DATE,SEASON
0,1,1,Cole,Toad,Sherbet Land,4,4,2021-09-20,0
1,1,1,Connor,Yoshi,Sherbet Land,2,4,2021-09-20,0
2,1,1,Cooper,Peach,Sherbet Land,1,4,2021-09-20,0
3,1,1,Triston,Bowser,Sherbet Land,3,4,2021-09-20,0
4,2,1,Cole,Toad,Kalimari Desert,4,4,2021-09-20,0
...,...,...,...,...,...,...,...,...,...
20674,6306,738,Cooper,Peach,Yoshi Valley,2,4,NaN,11
20675,6306,738,Connor,Yoshi,Yoshi Valley,3,4,NaN,11
20676,6306,738,Blake,Luigi,Yoshi Valley,4,4,NaN,11
20677,6307,738,Cooper,Peach,Toad's Turnpike,1,2,NaN,11


## 4 Player ELO

In [10]:
df = load_data_pd(
    sheet_name="data_main",
    sheet_id=os.environ["SHEET_ID"],
    usecols=[
        "UID",
        "SUID",
        "NAME",
        "CHARACTER",
        "MAP",
        "PLACE",
        "PLAYERS",
        "DATE",
        "SEASON",
    ],
)

df["DATE"] = pd.to_datetime(df["DATE"]).astype(str)

invalid, valid = validate_bad_uids(df=df, return_valid=True)

In [11]:
tdf = (
    valid[(valid["PLAYERS"] == 4) & (valid["SEASON"] == 11)]
    .drop(columns=["SUID", "CHARACTER", "PLAYERS", "SEASON", "MAP"])
    .sort_values(by=["UID", "PLACE"])
    .copy()
    .reset_index(drop=True)
)

pdf = (
    tdf.pivot(index="UID", columns="PLACE", values="NAME")
    .reset_index()
    .rename(columns={1: "1st", 2: "2nd", 3: "3rd", 4: "4th"})
    .merge(tdf[["UID", "DATE"]], on="UID", how="inner", validate="1:m")
    .groupby("UID")
    .first()
    .set_index("DATE")
    .reset_index()
    .rename(columns={"DATE": "date"})
)

exp_elo = MultiElo(score_function_base=2)
tracker = Tracker(elo_rater=exp_elo)
tracker.process_data(pdf)

rdf = tracker.get_current_ratings()

In [12]:
# filter_df = rdf[rdf["n_games"] > 1].copy()


filter_df = rdf.copy()
filter_df

,rank,player_id,n_games,rating
0,1,Cooper,176,1357.791886
1,2,Cole,49,1352.460872
2,3,Regan,76,1206.265419
3,4,Blake,47,1145.619577
4,5,Matt,68,1120.815867
5,6,Konnor,27,1104.817912
6,7,Connor,37,1100.460525
7,8,Luke,49,1059.298728
8,9,Chandler,11,1022.122982
9,10,Sam,4,968.800930


In [13]:
import altair as alt

track_df = tracker.get_history_df()

track_df = track_df[track_df["player_id"].isin(filter_df["player_id"])].reset_index(
    drop=True
)

alt.Chart(track_df).mark_line(point=True).encode(
    x="date:T",
    y=alt.Y("rating:Q", scale=alt.Scale(zero=False)),
    color="player_id:N",
).properties(width=1000, height=500, title="Season 11 ELO")

alt.Chart(...)

In [14]:
track_df.head()

,player_id,date,rating
0,Blake,2024-06-21 00:00:00,1037.090909
1,Blake,2024-06-21 00:00:00,1036.424120
2,Blake,2024-06-21 00:00:00,1036.466476
3,Blake,2024-06-21 00:00:00,1072.107255
4,Blake,2024-06-21 00:00:00,1069.675823


In [8]:
import plotly.express as px

In [15]:
fig = px.line(data_frame=track_df, x="date", y="rating", color="player_id", height=800, markers=True)

fig

In [16]:
fig.write_html(r"C:\Users\Cooper\sandbox\mkstream\assets\fig\s11_4p_elo.html")

In [ ]:
track_df.to_clipboard()